In [ ]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import scipy.io
%matplotlib inline

In [ ]:
class MyInfo:
    # Give names of class members.
    def __repr__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'
    def __str__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'

In [ ]:
folder = '/scratch/n.beaver/05_copus_isofreq_permalloy/copus_isofreq_permalloy_000.out/'
outfile_prefix = "05_copus_isofreq_permalloy_000"
tstep = 10e-12 # 10 ps timestep

In [ ]:
example1_raw = np.load(os.path.join(folder, 'm_full000000.npy'))

In [ ]:
example1_raw.shape

In [ ]:
example1 = example1_raw.squeeze()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.pcolormesh(example1[2]);

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.pcolormesh(example1[1]);

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.pcolormesh(example1[0]);

In [ ]:
del example1, example1_raw

In [ ]:
filenames = ['m_full{:06d}.npy'.format(i) for i in range(0, 8192)]

In [ ]:
%%time
cells_list = MyInfo()
cells_list.x = []
cells_list.y = []
cells_list.z = []
for i, filename in enumerate(filenames):
    filepath = os.path.join(folder, filename)
    dat_raw = np.load(filepath)
    dat = np.squeeze(dat_raw)
    cells_list.x.append(dat[0])
    cells_list.y.append(dat[1])
    cells_list.z.append(dat[2])
cells = MyInfo()
cells.x = np.stack(cells_list.x)
cells.y = np.stack(cells_list.y)
cells.z = np.stack(cells_list.z)

In [ ]:
cells.z.shape

In [ ]:
%%time
nt, nx, ny = cells.z.shape
reshape = MyInfo()
fft = MyInfo()
reshape.x = np.zeros((nx, ny, nt))
reshape.y = np.zeros((nx, ny, nt))
reshape.z = np.zeros((nx, ny, nt))
fft.x = np.zeros((nx, ny, nt), dtype=np.complex128)
fft.y = np.zeros((nx, ny, nt), dtype=np.complex128)
fft.z = np.zeros((nx, ny, nt), dtype=np.complex128)
for i in range(nx):
    for j in range(ny):
        reshape.x[i][j] = cells.x[:, i, j]
        reshape.y[i][j] = cells.y[:, i, j]
        reshape.z[i][j] = cells.z[:, i, j]
        fft.x[i][j] = np.fft.fft(reshape.x[i][j]/Msat)
        fft.y[i][j] = np.fft.fft(reshape.y[i][j]/Msat)
        fft.z[i][j] = np.fft.fft(reshape.z[i][j]/Msat)

In [ ]:
fft.x.shape

In [ ]:
fft_freq_Hz = np.fft.fftfreq(nt)/tstep

In [ ]:
tstep

In [ ]:
# mat_dict = {
#     "Msat": Msat,
#     "tstep": tstep,
#     "folder": folder,
#     'fft_freq_Hz': fft_freq_Hz
# }
# scipy.io.savemat(
#     outfile_prefix + "_fft.mat",
#     mat_dict,
#     long_field_names=True,
#     do_compression=True,
# )

In [ ]:
# %%time
# these files are 8 GB each
# np.save(outfile_prefix + "_fftx.npy", fft.x)
# np.save(outfile_prefix + "_ffty.npy", fft.y)
# np.save(outfile_prefix + "_fftz.npy", fft.z)

In [ ]:
%%time
fft_avg = MyInfo()
fft_avg.x = np.abs(fft.x).mean((0,1))
fft_avg.y = np.abs(fft.y).mean((0,1))
fft_avg.z = np.abs(fft.z).mean((0,1))

In [ ]:
mat_dict_avg = {
    "fftx_avg": fft_avg.x,
    "ffty_avg": fft_avg.y,
    "fftz_avg": fft_avg.z,
    "Msat": Msat,
    "tstep": tstep,
    "folder": folder,
    'fft_freq_Hz': fft_freq_Hz
}
scipy.io.savemat(
    outfile_prefix + "_fft_avg.mat",
    mat_dict_avg,
    long_field_names=True,
    do_compression=True,
)

In [ ]:
fft_freq_Hz

In [ ]:
ns = 1e-9
t_ns = np.linspace(0, tstep*nt, nt)/ns
f_nyquist = 2/(tstep)

In [ ]:
nt*tstep/ns

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(16,9))
# ax.plot(t_ns, cells.z[:, 0, 0]);
# ax.plot(t_ns, cells.z[:, 0, 1], zorder=0);
ax.plot(t_ns, cells.z[:, 1, 0], zorder=0);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del fig, ax

In [ ]:
GHz = 1e9
fft_freq_Hz.max()/GHz

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(fft_freq_Hz, fft_avg.z);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del fig, ax

In [ ]:
def closest_index(arr, val):
    if val > arr.max():
        raise ValueError("not in range: {} > {}".format(val, arr.max()))
    elif val < arr.min():
        raise ValueError("not in range: {} < {}".format(val, arr.min()))
    index = np.argmin(abs(arr - val))  # TODO: use binary search or something else?
    return index

In [ ]:
5*GHz

In [ ]:
i0 = closest_index(fft_freq_Hz, 0.0)
i_max = closest_index(fft_freq_Hz, 5*GHz)
fig, ax = plt.subplots(constrained_layout=True, figsize=(16*0.7,9*0.7))
ax.plot(fft_freq_Hz[i0:i_max]/GHz, fft_avg.z[i0:i_max]);
ax.set_title(folder)
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('FFT mag. ($M_z$/$M_s$), log scaling');
ax.axvline(1425e6/GHz, color='gray', linestyle='--', zorder=0, label='1425 MHz')
ax.axvline(2850e6/GHz, color='black', linestyle=':', zorder=0, label='2850 MHz')
ax.set_yscale('log')
ax.legend();

In [ ]:
fig.canvas.draw()

In [ ]:
fig.savefig(
    outfile_prefix + "_fft_Mz_norm_vs_freq.png",
#     bbox_inches='tight'
    dpi=300,
);

In [ ]:
plt.close(fig); del fig, ax